In [1]:
import molsysmt as msm
import numpy as np

In [2]:
molsys = msm.convert(msm.demo['TcTIM']['1tcd.msmpk'], to_form='molsysmt.MolSys')
output = msm.select(molsys, 'chain_id=="A" within 0.30 nm of chain_id=="B"')
true_output = np.array([ 89, 480, 527, 547, 550, 552, 554, 566, 723, 734])

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


ZeroDivisionError: division by zero